<a href="https://colab.research.google.com/github/Itsuki-Hamano123/optimizer_prac/blob/master/tsp/verification_quantum_and_googleor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化問題における量子コンピュータと古典コンピュータのアプローチの違い
現状の量子コンピュータと古典コンピュータ、それぞれどんな手順で最適化問題を解き、どんな解が得られるか実験

**去年、ITmediaに、量子コンピュータは巡回セールスマン問題が解けないのかというテーマで２つ記事が掲載されました。**<br>
[ITmedia記事１（https://www.itmedia.co.jp/news/articles/1906/03/news033.html）](https://www.itmedia.co.jp/news/articles/1906/03/news033.html)<br>
[ITmedia記事２（https://www.itmedia.co.jp/news/articles/1911/20/news149.html）](https://www.itmedia.co.jp/news/articles/1911/20/news149.html)<br>
***
2つ目の記事には、「自分の講演では、量子アニーリングで巡回セールスマン問題をやってはダメと伝えている。理由は、組合せ最適化問題の中でも難しい問題で、ガチガチの制約条件を守らないといけないから」という内容がありました。この記事では、主に量子コンピュータのハードから起因したノイズについてメインに語られていました。



一方で現状の量子コンピュータで、最適化問題を扱うためには、QUBO:(Quadratic Unconstrained Binary Optimization)と言われる「制約なし二次形式二値変数最適化」のフレームワークに落とし込まなければいけません。<br>
DeNAで開催された勉強では、**QUBOが起因して巡回セールスマン問題が解けない場合もある**ということを教わりました。<br>
そこで、今回は量子コンピュータハードのノイズでなく、そもそも量子コンピュータで制約条件つきの最適化問題を解く際の手順に着目していきたいと思います。

## 巡回セールスマン問題を解く手順
- **量子コンピュータ**
 1. 目的関数と制約条件を、それぞれQUBO（制約なし二次形式二値変数最適化）行列に落とし込む
 2. ラグランジュ未乗定数（ハイパーパラメータ）を導入し１つの目的関数に結合
 3. 量子コンピュータで、最適化問題用の量子アルゴリズム（量子アニーリングないしはQAOA）を使い解（局所解）を求める
 4. **制約条件が守られるようなラグランジュ未定乗数を見つけるまで２～３を繰り返す**
 ***
- **古典コンピュータ**
 1. 目的関数と制約条件を定義
 2. 制約付き最適化ソルバ（今回は動的計画法）に、目的関数と制約条件をそれぞれ渡し、解（局所解または大域最適解）を求める
 ***
 驚くべきことに、量子コンピュータで制約付き最適化問題を解こうとしたときには、手順2～3を何度も繰り返す必要があり、元の最適化問題を解くためにはラグランジュ未乗定数の最適化も行わないといけないことが分かります。<br>

実際に試してみます。
### 使用するライブラリ
- **座標（グラフ）の生成**
 - NetworkX
- **量子コンピュータのソルバ関連**
 - PyQUBO
 （現状の量子コンピュータ実機で計算してしまうとノイズが発生してしまうので、今回は古典コンピュータ上で量子アニーリングをシュミレートして解を求めます。）
- **古典コンピュータのソルバ関連**
 - Google OR-Tools（巡回セールスマン問題用のソルバがあるので使ってみます）


#### ライブラリのインストールと確認

In [0]:
!pip install pyqubo

     |████████████████████████████████| 4.7MB 5.4MB/s 
     |████████████████████████████████| 399kB 37.0MB/s 
  Created wheel for pyqubo: filename=pyqubo-0.4.0-cp36-none-any.whl size=40712 sha256=8424965601c9f44d55353effe465d581570d49336f953c151979763b61e7638f
  Stored in directory: /root/.cache/pip/wheels/77/b7/47/b989889699f788f4fecdc534bfddbb16df9a9a51f4e431b04d
Successfully built pyqubo


In [0]:
!python -m pip install --upgrade --user ortools

     |████████████████████████████████| 27.9MB 144kB/s 
     |████████████████████████████████| 1.3MB 45.7MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.


In [0]:
!pip show pyqubo
!pip show ortools
!pip show networkx

Name: pyqubo
Version: 0.4.0
Summary: PyQUBO allows you to create QUBOs or Ising models from mathematical expressions.
Home-page: https://github.com/recruit-communications/pyqubo
Author: Recruit Communications Co., Ltd.
Author-email: rco_pyqubo@ml.cocorou.jp
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: dwave-neal, dimod, six, numpy
Required-by: 
Name: ortools
Version: 7.5.7466
Summary: Google OR-Tools python libraries and modules
Home-page: https://developers.google.com/optimization/
Author: Google Inc
Author-email: lperron@google.com
License: Apache 2.0
Location: /root/.local/lib/python3.6/site-packages
Requires: protobuf, six
Required-by: 
Name: networkx
Version: 2.4
Summary: Python package for creating and manipulating graphs and networks
Home-page: http://networkx.github.io/
Author: Aric Hagberg
Author-email: hagberg@lanl.gov
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: decorator
Required-by: scikit-image, python-louvain, 

### 必要モジュールのimport、定義

In [0]:
# 必要モジュール
from pprint import pprint
import time
import networkx

In [0]:
def time_watch(func):
    """時間計測用デコレータ関数
    
    Parameters
    -----
    func : function
        実行したい関数

    """
    import functools
    import datetime
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.today()
        result = func(*args, **kwargs)
        end = datetime.datetime.today()
        print("実行時間:{time}".format(time=end - start))
        return result
    return wrapper

In [0]:
def explain(item, shows_private=False, shows_method=False):
    """
    与えた python オブジェクトの詳細を表示します。
    
    Notes
    -----
    引用元のサイト:https://qiita.com/halhorn/items/7b8351c5eafbfa28d768
    """
    print('EXPLAIN ------------------')
    print(item)
    print(type(item))
    print('ATTRIBUTES:')
    for d in dir(item):
        if d == 'type':
            continue
        if not shows_private and d.startswith('_'):
            continue
        attr = getattr(item, d)
        if not shows_method and (
                isinstance(attr, types.MethodType) or
                isinstance(attr, types.BuiltinMethodType) or
                isinstance(attr, types.CoroutineType) or
                isinstance(attr, types.FunctionType) or
                isinstance(attr, types.BuiltinFunctionType) or
                isinstance(attr, types.GeneratorType)
        ):
            continue
        print('{}:\t{}'.format(d, attr))

## 座標を定義